In [105]:
%cd '/content/drive/MyDrive/CSE250A/HW1'

/content/drive/MyDrive/CSE250A/HW1


In [106]:
def processing(file):
    
    word_count = {}
    f = open(file,'r')
    
    for line in f:
        line = line.split(" ")
        word_count[line[0]] = int(line[1])
    return word_count

In [107]:
def priorProbability(word_dict):
    word_prob = {}
    total  = sum(list(word_dict.values()))
    for key, value in word_dict.items():
        word_prob[key] = value / total
    
    return word_prob

In [108]:
#initial word count      
word_count = processing("hw1_word_counts_05.txt")
#initial word probabilities
word_prob = priorProbability(word_count)

In [109]:
sorted_word_probabilities = sorted(word_prob.items(), key = lambda x: x[1], reverse = True)


print("Top 15 most frequent words:")
for i in range(15):
    print(sorted_word_probabilities[i][0])


print("14 least frequent words:")
for i in range(-1, -15, -1):
    print(sorted_word_probabilities[i][0])

Top 15 most frequent words:
THREE
SEVEN
EIGHT
WOULD
ABOUT
THEIR
WHICH
AFTER
FIRST
FIFTY
OTHER
FORTY
YEARS
THERE
SIXTY
14 least frequent words:
TROUP
OTTIS
MAPCO
CAIXA
BOSAK
YALOM
TOCOR
SERNA
PAXON
NIAID
FOAMY
FABRI
CLEFT
CCAIR


In [110]:
def calcPosteriorProbs(possible_words, total):
  posterior_prob = {}
  for word, value in possible_words.items():
    posterior_prob[word] =  value/total
  return posterior_prob

In [169]:
def getPossibleLetters(possible_words, correct_guess, wrong_guess):
  possible_letters = set()
  for word in possible_words.keys():
    l = list(map(str, word))
    possible_letters.update(l)
  correct_letters = [l for l, v in correct_guess.items()]
  possible_letters = possible_letters - set(wrong_guess) - set(correct_letters)
  return possible_letters

In [112]:
def calcLetterProb(letters, posterior_probs):
  letter_predictive_prob ={}
  next_guess, val = None, 0

  for letter in letters:
    letter_predictive_prob[letter] = 0
    for word, value in posterior_probs.items():
      if letter in word:
        letter_predictive_prob[letter] += value
    if letter_predictive_prob[letter] > val:
      next_guess = letter
      val = letter_predictive_prob[letter]
  
  return letter_predictive_prob, next_guess, val

In [171]:
def getPossibleWords(word_probs, correct_guess, wrong_guess):
  possible_words = {}
  total = 0
  for word,prob in word_probs.items():
    sub = {}
    flag = 0
    for letter in wrong_guess:
      if letter in word:
        flag = 1
        break
    if flag:
      continue
    
    for i,ch in enumerate(word):
      sub[ch] = sub.get(ch,[]) + [i]
    for letter, value in correct_guess.items():
      if letter not in word:
        flag = 1
        break
      else:
        if sub[letter] != value:
          flag = 1
          break
    if flag:
      continue
    possible_words[word] = prob
    total += prob
  return possible_words, total


In [184]:
def predictNextLetter(word_count, word_prob, correct_guess, wrong_guess):
  
  possible_words, total_poss_prob  = getPossibleWords(word_prob, correct_guess, wrong_guess)
  
  word_posterior_prob = calcPosteriorProbs(possible_words, total_poss_prob)
  possible_letters = getPossibleLetters(possible_words, correct_guess, wrong_guess)
  letter_probs, next_guess, val = calcLetterProb(possible_letters, word_posterior_prob)

  return (next_guess, val)

In [174]:
samples = []
correct = []
wrong = []

for i in range(9):
  word = input("Enter correctly guessed:")
  samples.append(word)
  w_guess = input("Enter incorrectly guessed:")
  word_w = w_guess.split(',')
  if word_w[0] =='':
    wrong.append([])
  else:
    wrong.append(word_w)

Enter correctly guessed:-----
Enter incorrectly guessed:
Enter correctly guessed:-----
Enter incorrectly guessed:E,A
Enter correctly guessed:A---S
Enter incorrectly guessed:
Enter correctly guessed:A---S
Enter incorrectly guessed:I
Enter correctly guessed:--O--
Enter incorrectly guessed:A,E,M,N,T
Enter correctly guessed:-----
Enter incorrectly guessed:E,O
Enter correctly guessed:D--I-
Enter incorrectly guessed:
Enter correctly guessed:D--I-
Enter incorrectly guessed:A
Enter correctly guessed:-U---
Enter incorrectly guessed:A,E,I,O,S


In [175]:
print(samples)
print(wrong)

['-----', '-----', 'A---S', 'A---S', '--O--', '-----', 'D--I-', 'D--I-', '-U---']
[[], ['E', 'A'], [], ['I'], ['A', 'E', 'M', 'N', 'T'], ['E', 'O'], [], ['A'], ['A', 'E', 'I', 'O', 'S']]


In [206]:
for word in samples:
  s = {}
  for i,ch in enumerate(word):
    if ch != '-':
      s[ch] = s.get(ch,[])+[i]
  correct.append(s)
print(correct)

[{}, {}, {'A': [0], 'S': [4]}, {'A': [0], 'S': [4]}, {'O': [2]}, {}, {'D': [0], 'I': [3]}, {'D': [0], 'I': [3]}, {'U': [1]}, {}, {}, {'A': [0], 'S': [4]}, {'A': [0], 'S': [4]}, {'O': [2]}, {}, {'D': [0], 'I': [3]}, {'D': [0], 'I': [3]}, {'U': [1]}]


In [205]:
print('correctly guessed\tincorrectly guessed\t\tbest next guess\t\t\tProbability')
for i in range(9):
  next_guess, next_prob = predictNextLetter(word_count, word_prob, correct[i], wrong[i])
  print('{:30s}{:30s}{:30s}{:4.4f}'.format(samples[i],' '.join(wrong[i]), next_guess, next_prob))

correctly guessed	incorrectly guessed		best next guess			Probability
-----                                                       E                             0.5394
-----                         E A                           O                             0.5340
A---S                                                       E                             0.7715
A---S                         I                             E                             0.7127
--O--                         A E M N T                     R                             0.7454
-----                         E O                           I                             0.6366
D--I-                                                       A                             0.8207
D--I-                         A                             E                             0.7521
-U---                         A E I O S                     Y                             0.6270
